In [1]:
import pandas as pd

names=pd.read_csv('/Users/hubertcheung/Downloads/temp.csv')
names=names.dropna()
nameparse=names['name'].str.split()

nameparse = nameparse[nameparse.apply(lambda x: (isinstance(x, list) and len(x)>0))]
nameparse = nameparse[nameparse.apply(lambda x: 'LTD' not in x)]
nameparse

1          [LEE, WAI, HONG, &, NGAN, TIK, HAN, NDIYA, (JT)]
2                                         [CHAN, WAI, LUEN]
4                                    [LI, YEE, LIN, ELAINE]
5                  [CHU, MING, WAH, &, YU, LAI, PING, (JT)]
6                  [CHU, PO, LUN, &, LAU, SHUK, MING, (JT)]
                                 ...                       
2261121            [LIU, WING, KAI, &, LO, LAI, YING, (JT)]
2261122        [LEUNG, TIM, &, LEUNG, CHIM, YAN, IDY, (JT)]
2261123         [MOK, KANG, MING, &, LEE, CHI, CHING, (JT)]
2261124                                 [HAU, SIU, PO, 1/2]
2261125           [NG, KATHERINE, &, LI, CHOR, KEUNG, (JT)]
Name: name, Length: 2030377, dtype: object

In [2]:
with open('canton.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

canton = [line.strip() for line in lines]

In [9]:
with open('pinyin.txt','r',encoding='utf-8') as file:
    lines=file.readlines()

pinyin = [line.strip() for line in lines]

In [3]:
onename = nameparse[nameparse.astype(str).apply(lambda x: '&' not in x)]
#Those with only one element are weird and should be picked out
singleelement = onename[onename.apply(lambda x:len(x)==1)]
onename = onename[onename.apply(lambda x:len(x)!=1)]

In [6]:
df = pd.DataFrame(nameparse)

In [7]:
HKname = onename[onename.apply(lambda x: x[0] in canton and x[1] in canton)]
print(len(HKname))
HKname_v1 = onename[onename.apply(lambda x: x[0] in canton or x[1] in canton)]
print(len(HKname_v1))

554786
554786


In [10]:
MLname = onename[onename.apply(lambda x: x[0] in pinyin)]

In [33]:
import numpy as np
blur = np.intersect1d(HKname_v1.index, MLname.index)
firstpinyin = HKname_v1[blur]

In [21]:
possiblepinyin = firstpinyin[firstpinyin.apply(lambda x: x[1] in pinyin if len(x)==2 else x[1] in pinyin and x[2] in pinyin)]

In [34]:
MLname[~MLname.index.isin(blur)]

80                                        [HU, JACHIN]
102                                    [ZOU, ZHONGHUA]
198                                       [YANG, XIAO]
275                                    [ZHENG, JINLIN]
278                                  [MOU, CHIA, CHUN]
                              ...                     
2260692                                [HUANG, JIANRU]
2260732                                  [LAO, I, MUI]
2260753    [LIAO, SHIU, HSING, LUCY, "ADMINISTRATRIX"]
2260848                         [ZHANG, MIN, ISABELLE]
2261079                           [HUANG, HSIEH, LIEN]
Name: name, Length: 30175, dtype: object

In [76]:
def identify(name):
    return 'HK'

In [35]:
#Multiple name list
multiname = nameparse[nameparse.apply(lambda x: '&' in x)]

In [48]:
def count(a):
    count=0
    for item in a:
        if item == '&':
            count+=1
    return count

counting = nameparse.apply(lambda x: count(x))

In [63]:
def locate(a,n):
    count = 0
    for i,item in enumerate(a):
        if item == '&':
            count += 1
        if count == n:
            return i
    if n == 0:
        return 0
    if n == count(a):
        return len(a)

In [69]:
def clean(mylist):
    i = 1
    newlist = []
    consecutive = False
    for item in mylist:
        if item == '&':
            if not consecutive:
                newlist.append(item)
                consecutive = True
        else:
            newlist.append(item)
            consecutive = False
    if mylist[-1] == '&':
        mylist = mylist[:-1]
    return newlist

In [70]:
nameparse = nameparse.apply(lambda x: clean(x))

In [71]:
#This only works on multiple names.
def nameloop(namelist):
    namecount = count(namelist) #Notice that here namecount is actually the number of names - 1.
    origin = []
    for num in range(namecount):
        name = namelist[locate(namelist,num):locate(namelist,num+1)]
        if name[0] == '&':
            name = name[1:]
        origin.append(identify(name))
    check1 = {'HK','ML','F'}
    check2 = {'HK','ML'}
    check3 = {'HK','F'}
    check4 = {'ML','F'}
    check5 = {'HK'}
    check6 = {'ML'}
    check7 = {'F'}
    check = [check1,check2,check3,check4,check5,check6,check7]
    origin_set = set(origin)
    for item in check:
        if item == origin_set:
            return item
    #Directly return origin_set may incur problem about order of elements.

In [77]:
f = lambda x: identify(x) if count(x)==0 else ''
df['single'] = df['name'].apply(f)
        
g = lambda x: nameloop(x) if count(x)>0 else ''
df['multiple'] = df['name'].apply(g)

In [78]:
df

name single multiple
1        [LEE, WAI, HONG, &, NGAN, TIK, HAN, NDIYA, (JT)]            {HK}
2                                       [CHAN, WAI, LUEN]     HK         
4                                  [LI, YEE, LIN, ELAINE]     HK         
5                [CHU, MING, WAH, &, YU, LAI, PING, (JT)]            {HK}
6                [CHU, PO, LUN, &, LAU, SHUK, MING, (JT)]            {HK}
...                                                   ...    ...      ...
2261121          [LIU, WING, KAI, &, LO, LAI, YING, (JT)]            {HK}
2261122      [LEUNG, TIM, &, LEUNG, CHIM, YAN, IDY, (JT)]            {HK}
2261123       [MOK, KANG, MING, &, LEE, CHI, CHING, (JT)]            {HK}
2261124                               [HAU, SIU, PO, 1/2]     HK         
2261125         [NG, KATHERINE, &, LI, CHOR, KEUNG, (JT)]            {HK}

[2030377 rows x 3 columns]